In [47]:
pip install numpy 


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import Ridge

house_df = pd.read_csv('Housing.csv')

# 1. Cleaning up null values
house_df.dropna(inplace=True)

# 2a. Handle missing data
house_df.replace({'-': None}, inplace=True)

# 2b. Remove extreme outliers
for column in ['price', 'area']:
    q_low = house_df[column].quantile(0.01)
    q_hi  = house_df[column].quantile(0.99)
    house_df = house_df[(house_df[column] > q_low) & (house_df[column] < q_hi)]

# 3. One-Hot encoding
yn_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
house_df[yn_columns] = house_df[yn_columns].applymap(lambda x: 1 if x == 'yes' else 0)

# 4. Convert Categorical values to numerical
non_binary_cols = ['furnishingstatus']
house_df = pd.get_dummies(house_df, columns=non_binary_cols, drop_first=True)

# 5. Standardization/Normalization
scaler = MinMaxScaler()
numeric_cols = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
house_df[numeric_cols] = scaler.fit_transform(house_df[numeric_cols])

# 6. Deal with multicollinearity
# Using Ridge regression to handle potential multicollinearity
ridge = Ridge(alpha=1.0)
ridge.fit(house_df.drop('price', axis=1), house_df['price'])

house_df.to_csv('cleaned_house.csv', index=False)
cleaned_house = pd.read_csv('cleaned_house.csv')


/var/folders/dk/k7pw1l8s2hx5c3bnpxcc354c0000gn/T/ipykernel_13737/142119348.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  house_df[yn_columns] = house_df[yn_columns].applymap(lambda x: 1 if x == 'yes' else 0)


In [49]:
cleaned_house

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,1.000000,0.652131,0.6,1.0,1.000000,1,0,0,0,1,0.666667,1,True,False
1,0.966102,0.604559,0.6,0.0,0.333333,1,1,1,0,1,0.666667,1,False,False
2,0.957627,0.371655,0.4,0.5,1.000000,1,1,0,0,1,0.333333,1,False,True
3,0.943220,0.396432,0.6,1.0,0.333333,1,1,1,1,0,0.666667,0,True,False
4,0.898305,0.450942,0.6,0.5,0.333333,1,0,0,0,1,0.333333,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,0.025424,0.039643,0.4,0.0,0.333333,1,0,0,0,0,0.000000,0,False,True
517,0.025424,0.099108,0.6,0.0,0.333333,1,0,0,0,0,0.000000,0,False,True
518,0.025424,0.134787,0.2,0.0,0.000000,1,0,0,0,0,0.333333,0,False,True
519,0.008475,0.140733,0.8,0.0,0.333333,0,0,0,0,0,0.000000,0,False,True
